<h1 style="text-align:center;"> Graphene Model</h1>
<div style="height:100px;"></div>
<h3 style="text-align:center;"> Anoop</h3>

In [ ]:
import numpy as np
import scipy.linalg as spl

%matplotlib inline
import matplotlib
import matplotlib.pylab as plt

from sympy import Symbol,sin,cos,nsolve
from scipy.optimize import minimize
from scipy.optimize import root

from ipywidgets import interact, fixed
import ipywidgets as widgets

import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
from IPython.core.display import display, HTML

# The polling here is to ensure that plotly.js has already been loaded before
# setting display alignment in order to avoid a race condition.
display(HTML(
    '<script>'
        'var waitForPlotly = setInterval( function() {'
            'if( typeof(window.Plotly) !== "undefined" ){'
                'MathJax.Hub.Config({ SVG: { font: "STIX-Web" }, displayAlign: "center" });'
                'MathJax.Hub.Queue(["setRenderer", MathJax.Hub, "SVG"]);'
                'clearInterval(waitForPlotly);'
            '}}, 250 );'
    '</script>'
))
%config InlineBackend.figure_format = 'retina'

<!-- Equation and Algorithm-->
<!--<img src="sp2.png" style="display: block;margin:auto;width:40%;float:left;margin-top:60px">-->
<img src="unit_cell.png" style="display: block;margin:auto;width:60%;">

$$\vec{a1}=a\left(-\frac{1}{2}\vec{e_x}+\frac{\sqrt{3}}{2}\vec{e_y}\right), \vec{a2}=a\left(\frac{1}{2}\vec{e_x}+\frac{\sqrt{3}}{2}\vec{e_y}\right)$$

$$\vec{k1}=2\pi a^{-1}\left(-\vec{e_x}+\frac{1}{\sqrt{3}}\vec{e_y}\right), \vec{k2}=2\pi a^{-1}\left(\vec{e_x}+\frac{1}{\sqrt{3}}\vec{e_y}\right)$$

<h3 style="text-align:center;">Tight binding for multi-atomic Bravais lattice</h3>
$$\hat{H}=-\frac{1}{2}\nabla^2+\sum_{\vec{R}}U\left(\vec{r}-\vec{R}\right)=-\frac{1}{2}\nabla^2+\sum_{\vec{R}}\sum_{\vec{r_n}}U_0\left(\vec{r}-\vec{R}-\vec{r_n}\right)$$
$$\psi(\vec{r})=\sum_n\sum_{\vec{R}}e^{i\vec{k}\vec{R}}b_n\phi(\vec{r}-\vec{R}-\vec{r_n})$$

$$\hat{H}=-\frac{1}{2}\nabla^2+U(\vec{r}-\vec{r_m})+\sum_{n\ne m}\sum_{\vec{R}}U_0(\vec{r}-\vec{R}-\vec{r_n})$$
$$Eb_m=t\sum_{(n,\vec{R})\in\sigma_m}b_n e^{i\vec{k}\vec{R}}$$

$$(n,\vec{R})\rightarrow\text{(}n^{th}\text{ in unit cell }\vec{R}\text{)}$$
$$\sigma_m \rightarrow\text{( set of tuples belonging to the neighboring atoms of the (m,0)}\text{)} $$
<img src="sublattice.png" style="display: block;margin:auto;width:30%;">

### Graphene Band Structure

In [ ]:
a=1;t=-2.6; a0=[0,0]
a1=[-1.*a/2.,a*np.sqrt(3.)/2.]
a2=[1.*a/2.,a*np.sqrt(3.)/2.]
b1 = [1.*a/2.,-1*a*np.sqrt(3.)/2.]
b2 = [-1.*a/2.,-1*a*np.sqrt(3.)/2.]

sigma=[]
for i in range(2):
    if (i==0): sigma.append([[1,a1],[1,a2],[1,a0]])
    if (i==1): sigma.append([[0,b1],[0,b2],[0,a0]])
        
H = np.zeros((2, 2), dtype=np.complex)

def Hamiltonian(kx,ky):
    kvec=[kx,ky]
    for m in range(2):
        for n in range(2):
            H[m,n]=0
            if (m!=n):
                for i in range(3):
                    Rvec=sigma[m][i][1]
                    H[m,n] +=  np.complex(t*np.exp(1j*np.vdot(kvec,Rvec)))
    return H

In [ ]:
def plot_graphene_3D_N(n,xlim,ylim,resol):
    x = np.arange(xlim,ylim ,resol)
    y = np.arange(xlim,ylim ,resol)

    kx, ky  =  np.mgrid[xlim:ylim:resol, xlim:ylim:resol]
    # ZN = np.zeros_like(kx)

    ZN = np.zeros((n,kx.shape[0],kx.shape[0]))
    e=np.zeros(n)

    for i in range(kx.shape[0]):
        for j in range(kx.shape[1]):
            for m in range(n):
                H = Hamiltonian(kx[i,j],ky[i,j])
                evals, evecs = spl.eigh(H)
                ZN[m,i,j] = evals[m]

    data=[]
    for i in range(n):
        trace=go.Surface(
                x=x,
                y=y,
                z=ZN[i],
                showscale=True if i == 1 else False
            )
        data.append(trace)

    layout = go.Layout(
        title='Graphine-Numerical',
            margin=dict(
            l=65,
            r=50,
            b=65,
            t=90
        ),
    )
    fig2 = go.Figure(data=data, layout=layout)
    py.iplot(fig2, filename='3d-surface-graphene-numerical',show_link=False)
    return x,y,ZN

In [ ]:
n=2;lmin = -4*np.pi/3.;lmax = 4*np.pi/3.
x,y,ZN=plot_graphene_3D_N(n,lmin,lmax,0.1)

<p style="text-align:center;">Graphene nanoribbons</p>
<img src="zGNR.png" style="display: block;margin:auto;width:50%;">

### Graphene ZigZag Ribbon(zGNR)

In [ ]:
t=2.6; a0=0.; a1=1.

def sigma(m,kvec):
    ct=np.complex(t)
    nt=t*(1+np.exp(-1j*kvec*a1))
    pt=t*(1+np.exp(1j*kvec*a1))
    if (m%4==0) : pos1=m+1;val1=ct; pos2=m-1;val2=nt
    if (m%4==1) : pos1=m-1;val1=ct; pos2=m+1;val2=nt
    if (m%4==2) : pos1=m+1;val1=ct; pos2=m-1;val2=pt
    if (m%4==3) : pos1=m-1;val1=ct; pos2=m+1;val2=pt
    return pos1,val1,pos2,val2

def hamiltonian(N, kvec, edge=False, edgevalue=0.7*t):
    Eb = edgevalue
    H = np.zeros((2*N, 2*N), dtype=np.complex)
    for m in range(2*N):
        pos1,val1,pos2,val2 = sigma(m+1,kvec)
        if (pos1!=0) and (pos1!=2*N+1): H[m,pos1-1]=val1
        if (pos2!=0) and (pos2!=2*N+1): H[m,pos2-1]=val2
        if edge==True : H[0,0]=Eb;H[2*N-1,2*N-1]=Eb
    return H

In [ ]:
def band_struct(N,xlim, ylim ,resol,edge,edgevalue):
    x = np.arange(xlim, ylim ,resol)
    ZN = np.zeros((len(x),2*N))
    for i in range(len(x)):
        H = hamiltonian(N,x[i],edge,edgevalue)
        evals, evecs = spl.eigh(H)
        ZN[i,:] = evals
    return x, ZN


def plot_graphene_nanoribbons_Z(N, xlim, ylim,resol, edge, edgevalue):
    x,ZN=band_struct(N,xlim, ylim ,resol, edge, edgevalue)
    # #plotting
    data=[]
    for i in range(N):
        trace1=go.Scatter(
                x=x,
                y=ZN[:,i],
                name='-E'
            )
        data.append(trace1)
        trace2=go.Scatter(
                x=x,
                y=-ZN[:,i],
                name='+E'
            )
        data.append(trace2)

    layout = go.Layout(
        title='Graphine-nanoribbons(zGNR)',
        xaxis=dict(title = '<b>k</b>'),        
        yaxis=dict(title = '<b>E(k)</b>'),
        showlegend=False,
            margin=dict(
            l=65,
            r=50,
            b=65,
            t=90
        ),
    )
    fig2 = go.Figure(data=data, layout=layout)
    py.iplot(fig2, filename='3d-surface-graphene-numerical',show_link=False)

slider=widgets.SelectionSlider(
    options={'0': 0, '1': 1, '2': 2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, 
             '20':20, '30':30, '50':50, '80':80, '100':100},
    value=6,
    description='np',
    disabled=False,
    orientation='horizontal',
    readout=True,
)

xlim = -1*np.pi; ylim = np.pi; resol=0.1
def iplotz(i, edge, edgevalue):
    plot_graphene_nanoribbons_Z(i, xlim, ylim, resol, edge, edgevalue)
    
interact(iplotz,i=slider, edge=fixed(False), edgevalue=fixed(0.7*t));

### Graphene Armchair Ribbon(aGNR)

In [ ]:
t=2.6; a0=0.; a1=1.

def sigma(m,kvec):
    ct=np.complex(t)
    nt=t*np.exp(-1j*kvec*a1)
    pt=t*np.exp(1j*kvec*a1)
    if (m%4==0) : pos1=m-3;val1=ct; pos2=m+1;val2=ct; pos3=m-1;val3=nt
    if (m%4==1) : pos1=m+3;val1=ct; pos2=m+1;val2=ct; pos3=m-1;val3=ct
    if (m%4==2) : pos1=m-3;val1=ct; pos2=m-1;val2=ct; pos3=m+1;val3=ct
    if (m%4==3) : pos1=m+3;val1=ct; pos2=m-1;val2=ct; pos3=m+1;val3=pt
    return pos1, val1, pos2, val2, pos3, val3

def hamiltonian(N, kvec, edge=False, edgevalue=0.7*t):
    Eb = edgevalue
    H = np.zeros((2*N, 2*N), dtype=np.complex)
    for m in range(2*N):
        pos1,val1,pos2,val2,pos3,val3 = sigma(m+1,kvec)
        if (pos1>=1) and (pos1<=2*N): H[m,pos1-1]=val1
        if (pos2>=1) and (pos2<=2*N): H[m,pos2-1]=val2        
        if (pos3>=1) and (pos3<=2*N): H[m,pos3-1]=val3
        if edge==True : H[0,0]=Eb;H[1,1]=Eb;H[2*N-2,2*N-2]=Eb;H[2*N-1,2*N-1]=Eb
    return H

In [ ]:
def band_struct(N,xlim, ylim ,resol,edge,edgevalue):
    x = np.arange(xlim, ylim ,resol)
    ZN = np.zeros((len(x),2*N))
    for i in range(len(x)):
        H = hamiltonian(N,x[i],edge,edgevalue)
        evals, evecs = spl.eigh(H)
        ZN[i,:] = evals
    return x, ZN

In [ ]:
def plot_graphene_nanoribbons_Z(N, xlim, ylim,resol, edge, edgevalue):
    x,ZN=band_struct(N,xlim, ylim ,resol, edge, edgevalue)
    # #plotting
    data=[]
    for i in range(N):
        trace1=go.Scatter(
                x=x,
                y=ZN[:,i],
                name='-E'
            )
        data.append(trace1)
        trace2=go.Scatter(
                x=x,
                y=-ZN[:,i],
                name='+E'
            )
        data.append(trace2)

    layout = go.Layout(
        title='Graphine-nanoribbons(aGNR)',
        xaxis=dict(title = '<b>k</b>'),        
        yaxis=dict(title = '<b>E(k)</b>'),
        showlegend=False,
            margin=dict(
            l=65,
            r=50,
            b=65,
            t=90
        ),
    )
    fig2 = go.Figure(data=data, layout=layout)
    py.iplot(fig2, filename='aGNR',show_link=False)

slider=widgets.SelectionSlider(
    options={'0': 0, '1': 1, '2': 2, '3':3, '4':4, '5':5, '6':6, '7':7, '8':8, '9':9, 
             '20':20, '30':30, '50':50, '80':80, '100':100},
    value=6,
    description='np',
    disabled=False,
    orientation='horizontal',
    readout=True,
)

xlim = -1.*np.pi; ylim = np.pi; resol=0.1
def iplotz(i, edge, edgevalue):
    plot_graphene_nanoribbons_Z(i, xlim, ylim, resol, edge, edgevalue)
    
interact(iplotz,i=slider, edge=fixed(True), edgevalue=fixed(1.5*t));

<h2 style="text-align:center;"> Desity of states</h2>
<br/>
$$D(E)= \sum_m\delta(E-E_m)$$
$$\text{DOS}(E) = \frac{1}{c \sqrt{2\pi}} \sum_n{e^{-\frac{(E_n - E)^2}{2 c^2}}}$$

In [ ]:
E=ZN.flatten()
dos_n,bin_edges_n = np.histogram(E,80,normed=True)
dos_p,bin_edges_p = np.histogram(-E,80,normed=True)

In [ ]:

trace0 = go.Scatter(
    x = bin_edges_p,
    y = dos_p,
    mode = 'lines',
)
trace1 = go.Scatter(
    x = bin_edges_n,
    y = dos_n,
    mode = 'lines',
)

data_dos = [trace0,trace1]
layout_dos = dict(
    title = '<b>DOS-Graphene</b>',
    xaxis = dict(title = '<b>E</b>'),
    yaxis = dict(title = '<b>D(E)</b>'),
    showlegend=False,
    )

fig_dos = dict(data=data_dos, layout=layout_dos)
py.iplot(fig_dos, filename='dos-graphene',show_link=False)